# <p style="padding:10px;background-color:#87CEEB ;margin:10;color:#000000;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 10px 10px ;overflow:hidden;font-weight:50">Machine Learning Marathon finish time estimation</p>

The objective of this notebook is to create a machine learning model for predicting marathon finish times based on age, gender, and 5 km race time running at a marathon pace. Additional models will be developed to make predictions using 10 km or 21 km race times. The model is designed for runners who are already capable of completing a full marathon, as it will be trained on data from runners who have successfully finished the entire marathon distance. This predictive tool can be valuable for individuals preparing for a marathon, providing an estimate of their potential marathon finish time. This is particularly useful since many marathoners typically do not run more than 32-35 km during their training for a marathon.

### Loading the necessary Libraries

In [1]:
import pandas as pd
import numpy as np
import re 
from operator import itemgetter
import glob
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler,OneHotEncoder,RobustScaler,MinMaxScaler,OrdinalEncoder,FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from xgboost import XGBRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor


from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# <p style="padding:10px;background-color:#87CEEB ;margin:10;color:#000000;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 10px 10px ;overflow:hidden;font-weight:50">Read the Dataset</p>

Let's load data on finishers from the Boston Marathon for the years of 2015, 2016 and 2017. Dataset can be accessed via this link [Boston dataset](https://www.kaggle.com/datasets/rojour/boston-results).

In [2]:
df = pd.concat(map(pd.read_csv, glob.glob('dataset/*.csv')))
df = df[['Age','M/F','5K','Official Time']]

# <p style="padding:10px;background-color:#87CEEB ;margin:10;color:#000000;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 10px 10px ;overflow:hidden;font-weight:50">Clean data</p>

Let's clean the data by removing unwanted or empty values. For the 'Gender' attribute, only 'M' or 'F' values are allowed. The 'Age' should be any number between 18 and 100. The '5K time' should be represented in the string format 'h:mm:ss'.

In [3]:
print('Before transforming data\n')
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print('{} - {:.2f}%'.format(col,pct_missing))

def process_gender(value):
    if value in ('M', 'F'):
        return value
    else:
        return pd.NA  

def process_age(value):
    if (value >=18 and value <=100):
        return value
    else:
        return pd.NA  
    
def process_time(value):
    if bool(re.compile(r'^(\d):([0-5]\d):([0-5]\d)$').match((value))):
        return value
    else:
        return pd.NA 
    

df['M/F'] = df['M/F'].apply(process_gender)
df['Age'] = df['Age'].apply(process_age)
df['5K'] = df['5K'].apply(process_time)
df['Official Time'] = df['Official Time'].apply(process_time)

print('After transforming data\n')
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print('{} - {:.4f}%'.format(col,pct_missing))

Before transforming data

Age - 0.00%
M/F - 0.00%
5K - 0.00%
Official Time - 0.00%
After transforming data

Age - 0.0000%
M/F - 0.0000%
5K - 0.0029%
Official Time - 0.0000%


After applying transformations to the dataset, which resulted in only a small number of null values, these rows will be removed.

In [4]:
df.dropna(inplace = True)
df = df.drop_duplicates()

# <p style="padding:10px;background-color:#87CEEB ;margin:10;color:#000000;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 10px 10px ;overflow:hidden;font-weight:50">Define Transformers</p>

Let's develop a Custom Transformer for age categorization. The input will be an age number, and the output should represent age categories such as '18-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', and '80+'. 

Additionally, let's define a FunctionTransformer to convert the '5K time' from the string format 'h:mm:ss' to the numerical feature representing the total number of seconds.

In [5]:
class AgeTransformer(TransformerMixin, BaseEstimator):
    def __init__(self, input_cols, output_cols):
        self.input_cols = input_cols
        self.output_cols = output_cols

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.input_cols].apply(lambda x: pd.cut(x, bins = [0,34,39,44,49,54,59,64,69,74,79,100], labels=['18-34','35-39','40-44','45-49','50-54','55-59','60-64','65-69','70-74','75-79','80+']))
    
    def get_feature_names_out(self, feature_names):
        return self.output_cols

class convert_to_seconds(TransformerMixin, BaseEstimator):
    def __init__(self, input_cols, output_cols):
        self.input_cols = input_cols
        self.output_cols = output_cols

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.input_cols].applymap(lambda x: pd.to_timedelta(x).total_seconds() if bool(re.compile(r'^(\d):([0-5]\d):([0-5]\d)$').match(str(x))) else None)
    
    def get_feature_names_out(self, feature_names):
        return self.output_cols

# <p style="padding:10px;background-color:#87CEEB ;margin:10;color:#000000;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 10px 10px ;overflow:hidden;font-weight:50">Preprocessing Pipeline</p>

Let's define a ColumnTransformer, which will be utilized for transforming each column. Within the ColumnTransformer, pipelines are employed to make column transformations flexible and prevent data leakage during cross-validation.

In [6]:
X = df.drop(columns=['Official Time'],axis=1)

enc = convert_to_seconds(input_cols=['Official Time'], output_cols=['Official Time'])
Y = enc.transform(df)

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2)

preprocessors = ColumnTransformer([
    
    ('age_pipeline',Pipeline([
        ('age_transformed',AgeTransformer(input_cols=['Age'], output_cols=['Age'])),
        ('age_encoder',OneHotEncoder(sparse_output = False))       
        ]),["Age"]),
    
 
    ('gender_pipeline',Pipeline([('gender_encoder',OrdinalEncoder())]), ['M/F']),
    
    
    ('5K_pipeline',Pipeline([
        ('time_transformed',convert_to_seconds(input_cols=["5K"], output_cols=["5K"])),   
        ('minmax_scaler',MinMaxScaler())     
        ]),["5K"])
    
    ])

# X_train_transformed  = preprocessors.fit_transform(X_train)
# X_train_transformed_names = preprocessors.get_feature_names_out()
# X_train_transformed = pd.DataFrame(X_train_transformed, columns = X_train_transformed_names)

# X_test_transformed = preprocessors.transform(X_test)
# X_test_transformed_names = preprocessors.get_feature_names_out()
# X_test_transformed = pd.DataFrame(X_test_transformed, columns = X_test_transformed_names)


# <p style="padding:10px;background-color:#87CEEB ;margin:10;color:#000000;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 10px 10px ;overflow:hidden;font-weight:50">RandomizedSearchCV</p>

In this section, I will create pipelines that include a preprocessor and an ML model. These pipelines will be utilized in combination with predefined parameters to evaluate each model using RandomizedSearchCV.

In [7]:
pipe_lasso = Pipeline([   
    ('preprocessors', preprocessors),
    ('LASSO', Lasso())
])

pipe_ridge = Pipeline([   
    ('preprocessors', preprocessors),
    ('RIDGE', Ridge())
])

pipe_knn = Pipeline([   
    ('preprocessors', preprocessors),
    ('KNN', KNeighborsRegressor())
])

pipe_svr = Pipeline([
   
    ('preprocessors', preprocessors),
    ('SVR', SVR())
])

pipe_et = Pipeline([
   
    ('preprocessors', preprocessors),
    ('ET', ExtraTreesRegressor())
])

pipe_dt = Pipeline([
   
    ('preprocessors', preprocessors),
    ('DT', DecisionTreeRegressor())
])

pipe_rf = Pipeline([
   
    ('preprocessors', preprocessors),
    ('RF', RandomForestRegressor())
])


pipe_ada = Pipeline([
   
    ('preprocessors', preprocessors),
    ('ADA', AdaBoostRegressor())
])

pipe_gbr = Pipeline([
   
    ('preprocessors', preprocessors),
    ('GBR', GradientBoostingRegressor())
])

pipe_lgbr = Pipeline([
   
    ('preprocessors', preprocessors),
    ('LGBR', lgb.LGBMRegressor())
])

pipe_cat = Pipeline([
   
    ('preprocessors', preprocessors),
    ('CAT', CatBoostRegressor())
])

pipe_xgb = Pipeline([
   
    ('preprocessors', preprocessors),
    ('XGB', XGBRegressor())
])


lasso_param_grid = [{'LASSO__alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,0,1,5,10,20,30,35,40,45,50,55,100]
                
                    }]

ridge_param_grid = [{'RIDGE__alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,0,1,5,10,20,30,35,40,45,50,55,100]
                
                    }]


knn_param_grid = [{                 
                    'KNN__n_neighbors': [3, 5, 7, 9],
                    'KNN__weights': ['uniform', 'distance'],
                    'KNN__p': [1, 2],             
                    }]


et_param_grid = [{                 
                    'ET__n_estimators': [50, 100, 200],
                    'ET__max_depth': [None, 5, 10, 15],
                    'ET__min_samples_split': [2, 5, 10],
                    'ET__min_samples_leaf': [1, 2, 4],
                    'ET__max_features': [1.0, 'sqrt', 'log2']       
                    }]

dt_param_grid = [{                 
                    'DT__max_depth': [None, 5, 10, 15],
                    'DT__min_samples_split': [2, 5, 10],
                    'DT__min_samples_leaf': [1, 2, 4]          
                    }]

rf_param_grid = [{                 
                    'RF__n_estimators': [50, 100, 200],
                    'RF__max_features': [1.0, 'sqrt', 'log2'],
                    'RF__max_depth': [None, 10, 20, 30],
                    'RF__min_samples_split': [2, 5, 10],
                    'RF__min_samples_leaf': [1, 2, 4],
                    'RF__bootstrap': [True, False]               
                    }]

ada_param_grid = [{                 
                    'ADA__n_estimators': [50, 100, 200],
                    'ADA__learning_rate': [0.01, 0.1, 1.0],
                    'ADA__loss': ['linear', 'square', 'exponential']            
                    }]

gbr_param_grid = [{                 
                    'GBR__n_estimators': [50, 100, 200],
                    'GBR__learning_rate': [0.01, 0.1, 1.0],
                    'GBR__max_depth': [3, 5, 7],
                    'GBR__subsample': [0.8, 0.9, 1.0]          
                    }]

lgbr_param_grid = [{                 
                    'LGBR__n_estimators': [50, 100, 200],
                    'LGBR__learning_rate': [0.01, 0.1, 1.0],
                    'LGBR__max_depth': [3, 5, 7],
                    'LGBR__subsample': [0.8, 0.9, 1.0],
                    'LGBR__colsample_bytree': [0.8, 0.9, 1.0]         
                    }]

cat_param_grid = [{                 
                    'CAT__iterations': [50, 100, 200],
                    'CAT__learning_rate': [0.01, 0.1, 1.0],
                    'CAT__depth': [3, 5, 7],
                    'CAT__l2_leaf_reg': [1, 3, 5],
                    'CAT__subsample': [0.8, 0.9, 1.0]        
                    }]


xgb_param_grid = [{                 
                    'XGB__learning_rate': [0.01, 0.1, 0.2],
                    'XGB__n_estimators': [50, 100, 200],
                    'XGB__max_depth': [3, 5, 7],
                    'XGB__subsample': [0.8, 0.9, 1.0],
                    'XGB__colsample_bytree': [0.8, 0.9, 1.0]             
                    }]


lasso_grid_search = RandomizedSearchCV(estimator=pipe_lasso,
        param_distributions=lasso_param_grid,
        scoring='neg_mean_absolute_error',
        cv=5,n_iter=20,n_jobs=-1)

ridge_grid_search = RandomizedSearchCV(estimator=pipe_ridge,
        param_distributions=ridge_param_grid,
        scoring='neg_mean_absolute_error',
        cv=5,n_iter=20,n_jobs=-1)

knn_grid_search = RandomizedSearchCV(estimator=pipe_knn,
        param_distributions=knn_param_grid,
        scoring='neg_mean_absolute_error',
        cv=5,n_iter=20,n_jobs=-1)

et_grid_search = RandomizedSearchCV(estimator=pipe_et,
        param_distributions=et_param_grid,
        scoring='neg_mean_absolute_error',
        cv=5,n_iter=20,n_jobs=-1)

dt_grid_search = RandomizedSearchCV(estimator=pipe_dt,
        param_distributions=dt_param_grid,
        scoring='neg_mean_absolute_error',
        cv=5,n_iter=20,n_jobs=-1)

rf_grid_search = RandomizedSearchCV(estimator=pipe_rf,
        param_distributions=rf_param_grid,
        scoring='neg_mean_absolute_error',
        cv=5,n_iter=20,n_jobs=-1)

ada_grid_search = RandomizedSearchCV(estimator=pipe_ada,
        param_distributions=ada_param_grid,
        scoring='neg_mean_absolute_error',
        cv=5,n_iter=20,n_jobs=-1)

gbr_grid_search = RandomizedSearchCV(estimator=pipe_gbr,
        param_distributions=gbr_param_grid,
        scoring='neg_mean_absolute_error',
        cv=5,n_iter=20,n_jobs=-1)

lgbr_grid_search = RandomizedSearchCV(estimator=pipe_lgbr,
        param_distributions=lgbr_param_grid,
        scoring='neg_mean_absolute_error',
        cv=5,n_iter=20,n_jobs=-1)

cat_grid_search = RandomizedSearchCV(estimator=pipe_cat,
        param_distributions=cat_param_grid,
        scoring='neg_mean_absolute_error',
        cv=5,n_iter=20,n_jobs=-1)

xgb_grid_search = RandomizedSearchCV(estimator=pipe_xgb,
        param_distributions=xgb_param_grid,
        scoring='neg_mean_absolute_error',
        cv=5,n_iter=20,n_jobs=-1)

# <p style="padding:10px;background-color:#87CEEB ;margin:10;color:#000000;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 10px 10px ;overflow:hidden;font-weight:50">Train model and Save results to a text file</p>

Let's continue with model training. 

In [8]:
grids = [lasso_grid_search,
         ridge_grid_search,
         knn_grid_search,
         et_grid_search,
         dt_grid_search,
         rf_grid_search,
         ada_grid_search,
         gbr_grid_search,
         lgbr_grid_search,
         cat_grid_search,
         xgb_grid_search
         ]

for pipe in grids:
    pipe.fit(X_train,y_train)
    print('Finished training {}'.format(pipe.estimator.steps[-1][0]))

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 17 is smaller than n_iter=20. Running 17 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.533e+10, tolerance: 3.928e+07
  model = cd_fast.enet_coordinate_descent(
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 17 is smaller than n_iter=20. Running 17 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Finished training LASSO


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.08514e-16): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 16 is smaller than n_iter=20. Running 16 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Finished training RIDGE
Finished training KNN


...

Finished training ADA


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Finished training GBR


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Finished training LGBR
0:	learn: 2304.5035067	total: 156ms	remaining: 15.4s
1:	learn: 2144.0675449	total: 169ms	remaining: 8.27s
2:	learn: 2001.0008833	total: 181ms	remaining: 5.87s
3:	learn: 1877.9487580	total: 195ms	remaining: 4.68s
4:	learn: 1769.9191719	total: 208ms	remaining: 3.95s
5:	learn: 1674.8033334	total: 221ms	remaining: 3.47s
6:	learn: 1593.0412935	total: 234ms	remaining: 3.1s
7:	learn: 1523.4069669	total: 246ms	remaining: 2.83s
8:	learn: 1462.8484155	total: 259ms	remaining: 2.62s
9:	learn: 1412.3252407	total: 271ms	remaining: 2.44s
10:	learn: 1368.5351587	total: 283ms	remaining: 2.29s
11:	learn: 1331.0079804	total: 295ms	remaining: 2.17s
12:	learn: 1300.6577880	total: 307ms	remaining: 2.06s
13:	learn: 1274.4904589	total: 320ms	remaining: 1.97s
14:	learn: 1252.8140242	total: 334ms	remaining: 1.89s
15:	learn: 1235.1547046	total: 347ms	remaining: 1.82s
16:	learn: 1219.8140786	total: 359ms	remaining: 1.75s
17:	learn: 1206.9582958	total: 374ms	remaining: 1.7s
18:	learn: 1196.3

Now I will save results of the training and model evaluations will in txt and csv file.

In [28]:
    
grid_dict = {
                0: 'lasso',
                1: 'ridge',
                2: 'knn',
                3: 'et',
                4: 'dt',
                5: 'rf',
                6: 'ada',
                7: 'gbr',
                8: 'lgbr',
                9:'cat',
                10:'xgb'
                }

results = []
for i, model in enumerate(grids):
    print('{} Test metric: {}'.format(grid_dict[i],
    model.score(X_test,y_test)))
    print('{} Best Params: {}'.format(grid_dict[i], model.best_params_))
    y_true, y_pred = y_test, model.best_estimator_.predict(X_test),
    
    results.append({    
        'iterator': i,
        'model': grid_dict[i],
        'best_params': model.best_params_,
        'best_score': model.best_score_,
        'mae': mean_absolute_error(y_true,y_pred),
        'mse': mean_squared_error(y_true,y_pred),
        'rmse': np.sqrt(mean_squared_error(y_true,y_pred)),
        'r2': r2_score(y_true,y_pred)
        })
    
results = sorted(results, key=itemgetter('best_score'), reverse=True) 

 
with open('results/5K_grid_search_results.txt', 'w') as file:
    for result in results:
        file.write(f"Model: {result['model']}\n")
        file.write("\n")
        
        file.write("Best Parameters:\n")
        for key, value in result['best_params'].items():
            file.write(f"{key}: {value}\n")
        file.write("\n")
        
        file.write(f"Best Score: {result['best_score']}\n")
        file.write(f"mae: {result['mae']}\n")
        file.write(f"mse: {result['mse']}\n")
        file.write(f"rmse: {result['rmse']}\n")
        file.write(f"r2: {result['r2']}\n")
        file.write("\n")
        file.write("------------------------------------------------------------------------------\n")

models_scores = pd.DataFrame(results)
models_scores = models_scores[['model','mae','mse','rmse','r2']]
models_scores.to_csv('results/5K_models_scores.csv')

lasso Test metric: -831.474751537522
lasso Best Params: {'LASSO__alpha': 1e-15}
ridge Test metric: -831.4747515375153
ridge Best Params: {'RIDGE__alpha': 0}
knn Test metric: -867.9618955526126
knn Best Params: {'KNN__weights': 'uniform', 'KNN__p': 2, 'KNN__n_neighbors': 9}
et Test metric: -834.8307252454463
et Best Params: {'ET__n_estimators': 200, 'ET__min_samples_split': 10, 'ET__min_samples_leaf': 2, 'ET__max_features': 1.0, 'ET__max_depth': 10}
dt Test metric: -842.0657603170444
dt Best Params: {'DT__min_samples_split': 10, 'DT__min_samples_leaf': 4, 'DT__max_depth': 5}
rf Test metric: -851.7270905376032
rf Best Params: {'RF__n_estimators': 50, 'RF__min_samples_split': 10, 'RF__min_samples_leaf': 2, 'RF__max_features': 'log2', 'RF__max_depth': 10, 'RF__bootstrap': False}
ada Test metric: -882.1339456908137
ada Best Params: {'ADA__n_estimators': 100, 'ADA__loss': 'linear', 'ADA__learning_rate': 0.01}
gbr Test metric: -827.26930140518
gbr Best Params: {'GBR__subsample': 0.9, 'GBR__n_

Let's see how did models performed on the test set 

In [29]:
print(models_scores)

    model         mae           mse         rmse        r2
0     xgb  818.759750  1.263474e+06  1124.043537  0.796820
1     gbr  827.269301  1.261699e+06  1123.253784  0.797106
2     cat  827.489897  1.261339e+06  1123.093665  0.797163
3    lgbr  828.361916  1.259648e+06  1122.340284  0.797435
4   ridge  831.474752  1.265864e+06  1125.106228  0.796436
5   lasso  831.474752  1.265864e+06  1125.106228  0.796436
6      et  834.830725  1.269443e+06  1126.695656  0.795860
7      dt  842.065760  1.294528e+06  1137.773153  0.791826
8      rf  851.727091  1.312969e+06  1145.848759  0.788861
9     knn  867.961896  1.394560e+06  1180.914742  0.775740
10    ada  882.133946  1.395814e+06  1181.445827  0.775538


Overall, it appears that all models are performing well. We can observe R2 values within the range of 0.77-0.80, and MAE falls between 819-882 seconds (13.60 - 14.70 minutes), which is satisfactory given that we are attempting to predict marathon finish time based on age, gender, and 5k race time. If the model were trained with additional data, such as times for 10km and 21km, I anticipate that its performance would further improve.

# <p style="padding:10px;background-color:#87CEEB ;margin:10;color:#000000;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 10px 10px ;overflow:hidden;font-weight:50">Saving best model and making prediction</p>

Saving model

In [32]:
import joblib

best_model = grids[results[0]['iterator']]
print('Found best model {}'.format(best_model.estimator.steps[-1][0]))

joblib.dump(best_model, 'models/5K_best_model.pkl')

Found best model XGB


['models/5K_best_model.pkl']

Loading the model and making predictions for myself! 😁 (Even though I can typically run 5 km faster, this would be my expected time during longer runs)

In [33]:
from datetime import timedelta

best_model_loaded = joblib.load("models/5K_best_model.pkl") 

# Testing mysef
d = {'Age': [30], 'M/F': ['M'], '5K': ['0:23:00']}
test = pd.DataFrame(data=d)

predictions = best_model.best_estimator_.predict(test)
print('My predicted marathon time in hh:mm:ss is:', timedelta(seconds=int(predictions)))

# Testing mysef as female
d = {'Age': [30], 'M/F': ['F'], '5K': ['0:23:00']}
test = pd.DataFrame(data=d)

predictions = best_model.best_estimator_.predict(test)
print('My predicted marathon time in hh:mm:ss is:', timedelta(seconds=int(predictions)))

My predicted marathon time in hh:mm:ss is: 3:30:11
My predicted marathon time in hh:mm:ss is: 3:28:46


The current predictions from the model are reasonably accurate, with my best marathon time around 3:40:00. Considering this, training the model using either the 10 km time or the 21 km time may lead to even better results. Additionally, an interesting observation is that changing the gender from male to female seems to improve the predicted outcomes. This aligns with known information on the subject, as explored in the Exploratory Data Analysis (EDA) notebook, suggesting that females generally exhibit better endurance than males.

# Conclusion

A machine learning model has been successfully developed for predicting marathon finish times, leveraging features such as age, gender, and 5 km race time. The results demonstrate the model's effectiveness. Future efforts will involve the development of additional models to enhance predictive capabilities using alternative features, such as 10 km or 21 km race times. It is anticipated that these extended models will further improve the accuracy and precision of predictions for marathon finish times.